In [31]:
import nltk
import pandas as pd
import numpy as np
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict
from scipy.special import softmax
from collections import Counter
import json
from keywords_utils import *
import pandas as pd
import json
import os


In [2]:
DATA_PATH = '../../data/wiki'
KEYWORDS_PATH = '../../data'
OUT_DIR = '..'

In [33]:
train = pd.read_csv(f'{DATA_PATH}/train_original_split.csv')
with open('../../../HateXplain/Data/dataset.json', 'r') as f:
    exp = json.load(f)
    f.close()

In [35]:
# preprocess hatexplain
list_of_targets = []
sentences = []
targets_maj = []

for k, v in exp.items():
    annotations = v['annotators']
    
    
    all_anno = tuple(x['target'] for x in annotations )
    list_of_targets.append(all_anno)
    c = Counter([x for y in list(all_anno) for x in y])
    ts = [x for x, count in c.items() if count >= 2]
    
    # removing posts without targets
    # if 'None' not in ts:
    targets_maj.append(ts)
    sentences.append(v['post_tokens'])


In [36]:
def reduce_targets(list_of_targets):
    if ('Islam' in list_of_targets or 'mus' in list_of_targets) and ('Jewish' in list_of_targets or 'jew' in list_of_targets):
        return 'Both'
    elif ('Islam' in list_of_targets or 'mus' in list_of_targets):
        return 'Islam'
    elif ('Jewish' in list_of_targets or 'jew' in list_of_targets):
        return 'Jewish'
    else:
        return 'Neither'
exp_df = pd.DataFrame({'text': [' '.join(x) for x in sentences], 'targets': targets_maj})
exp_df['target_reduced'] = exp_df.apply(lambda row: reduce_targets(row['targets']),axis=1)

In [37]:
exp_df.to_csv('preprocessed_explain.csv', index=False)

In [38]:
# preprocess learning from the worst
def expand_targets_learning(targets):
    splitting_classes = ['asi.wom','bla.wom','non.white.wom','gay.man','asi.man','indig.wom','mus.wom','bla.man','gay.wom']
    list_of_targets_flat = []
    if targets in splitting_classes:
        if len(targets.split('.')) > 2:
            list_of_targets_flat += ['.'.join(targets.split('.')[:-1]), targets.split('.')[-1]]
        else:
            list_of_targets_flat += targets.split('.')
    else:
        list_of_targets_flat.append(targets)
    return list_of_targets_flat

    

with open('../../../Dynamically-Generated-Hate-Speech-Dataset/Dynamically Generated Hate Dataset v0.2.3.csv', 'r') as f:
    learning_df = pd.read_csv(f)

learning_df = learning_df[learning_df['split']=='train']
learning_df['target_reduced'] = learning_df.apply(lambda row: reduce_targets(expand_targets_learning(row['target'])), axis=1)

In [40]:
learning_df[['text', 'target_reduced']].to_csv('preprocessed_learning.csv', index=False)

In [49]:
dataset = 'learning'
train = pd.read_csv(f'./preprocessed_{dataset}.csv')
keywords, inverse_stem_lookup= gen_keywords(train, p=0.7, tail_cutoff=2, col='target_reduced', txt='text')

out_dict = {}
for k, v in keywords.items():
    pre_stemmed = []
    for w in v:
        pre_stemmed += inverse_stem_lookup[w]
    out_dict[k] = pre_stemmed

print(f'---Writing Keywords to keywords_{dataset}.txt')
print(f'Keys: {out_dict.keys()}')
with open(f'./keywords_{dataset}.txt', 'w') as f:
    json.dump(out_dict, f)
f.close()

---Number of Classes---
3
---Building Inverse Stemming Lookup---


100%|██████████| 446403/446403 [00:03<00:00, 111654.68it/s]


---Vocab Size Post Stemming---
16585
---Vocab Size with minimum frequency 2---
11774
---Number of Keywords per Class with Threshold 0.7---
Neither: 10801
Jewish: 28
Islam: 12
---Writing Keywords to keywords_learning.txt
Keys: dict_keys(['Neither', 'Jewish', 'Islam'])


In [44]:
learning_df[learning_df['target']=='jew']

,Unnamed: 0,acl.id,X1,text,label,type,target,level,split,round.base,annotator,round,acl.id.matched,target_reduced
1294,1329,acl11193,1329,"The smell of jews is digusting, it's like a pi...",hate,derogation,jew,original,train,2,CAgNlUizNm,2a,acl11194,Jewish
1300,1335,acl11199,1335,idk about you but I believe jews are controlli...,hate,derogation,jew,original,train,2,CAgNlUizNm,2a,acl11200,Jewish
1359,1394,acl11251,1394,There's just something about kikes that I don'...,hate,derogation,jew,original,train,2,CAgNlUizNm,2a,acl11252,Jewish
1452,1487,acl11335,1487,Not buying from nasty kike-owned stores again....,hate,derogation,jew,perturbation,train,2,CAgNlUizNm,2b,acl20643,Jewish
1463,1498,acl11345,1498,THIS IS THE REASON I WAKE UP THE THE MORNING F...,hate,support,jew,perturbation,train,2,QiOKkCi7F8,2b,acl11344,Jewish
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31567,31651,acl38484,31651,WITNESS what has been done ! their lives are w...,hate,derogation,jew,perturbation,train,4,LqLKTtrOmx,4b,acl39738,Jewish
31570,31654,acl38487,31654,"#blind, for not seeing how this #pandemic is o...",hate,derogation,jew,perturbation,train,4,GNZuCtwed3,4b,acl39741,Jewish
31635,31719,acl38545,31719,Russian tourist offered employee $1 million to...,hate,animosity,jew,perturbation,train,4,cYKSVBW4HD,4b,acl39791,Jewish
31809,31893,acl38701,31893,"i do lmao, me and him share the same ""opinion""...",hate,derogation,jew,original,train,4,QiOKkCi7F8,4a,acl38702,Jewish


In [28]:
df['tokens_full'] = df.apply(lambda x: word_tokenize(x[txt]), axis=1)
df['tokens_clean'] = df.apply(lambda x: [w.lower() for w in x['tokens_full'] if w not in stop_words and w.isalnum()], axis=1)
df['tokens_stem'] = df.apply(lambda x: [ps.stem(w) for w in x['tokens_clean']], axis=1)
df

,text,label,tokens_full,tokens_clean,tokens_stem
0,:If I may butt in I've spent the last 1/4 hour...,0,"[:, If, I, may, butt, in, I, 've, spent, the, ...","[if, i, may, butt, i, spent, last, hour, follo...","[if, i, may, butt, i, spent, last, hour, follo..."
1,On my you will find the apology that I owe you...,0,"[On, my, you, will, find, the, apology, that, ...","[on, find, apology, i, owe, shuffles, feet, lo...","[on, find, apolog, i, owe, shuffl, feet, look,..."
2,"Yep, that's Twin cities from which this articl...",0,"[Yep, ,, that, 's, Twin, cities, from, which, ...","[yep, twin, cities, article, originated]","[yep, twin, citi, articl, origin]"
3,See? I was right! ;-),0,"[See, ?, I, was, right, !, ;, -, )]","[see, i, right]","[see, i, right]"
4,` Thanks for fixing the spelling error in ``pr...,0,"[`, Thanks, for, fixing, the, spelling, error,...","[thanks, fixing, spelling, error, propaganda, ...","[thank, fix, spell, error, propaganda, i, ca, ..."
...,...,...,...,...,...
23134,` kys {| style=``background-color: #fdffe7; bo...,0,"[`, kys, {, |, style=, ``, background-color, :...","[kys, fdffe7, border, 1px, solid, fceb92, 2, m...","[ky, fdffe7, border, 1px, solid, fceb92, 2, mi..."
23135,"Yeah, I realized I created a duplicate ID. Sor...",0,"[Yeah, ,, I, realized, I, created, a, duplicat...","[yeah, i, realized, i, created, duplicate, id,...","[yeah, i, realiz, i, creat, duplic, id, sorri,..."
23136,` :::Yeah and in the earlier sentence I'd rewo...,0,"[`, :, :, :, Yeah, and, in, the, earlier, sent...","[yeah, earlier, sentence, i, reword, fever, pi...","[yeah, earlier, sentenc, i, reword, fever, pitch]"
23137,Why oh why... You removed the trolls ANI secti...,0,"[Why, oh, why, ..., You, removed, the, trolls,...","[why, oh, you, removed, trolls, ani, section, ...","[whi, oh, you, remov, troll, ani, section, drm..."


In [52]:
tokens = [x for y in df['tokens_stem'] for x in y]
count = Counter(tokens)
count_class = Counter(df['label'])

VOCAB_SIZE = len(count.keys())
NUM_OF_TOKENS = sum(count.values())
NUM_OF_CLASSES = len(count_class.keys())
NUM_OF_RECORDS = sum(count_class.values())

In [36]:
c = 0
for k, v in count.items():
    if v == 1:
        c +=1
        
len(count)

40553

In [47]:
sum(count_class.values())
len(df)

23139

In [55]:
likelihood = np.zeros((VOCAB_SIZE, NUM_OF_CLASSES))
prior = np.zeros((NUM_OF_CLASSES,1))
norm = np.zeros((VOCAB_SIZE, 1))

In [62]:
vocab = list(count.keys())
classes = list(count_class.keys())
for k, v in count_class.items():
    sub_df = df[df['label']==k]
    sub_tokens = [x for y in sub_df['tokens_stem'] for x in y]
    prior[classes.index(k), 0] = v / len(df)
    sub_token_count = Counter(sub_tokens)
    for s_token, s_count in sub_token_count.items():
        likelihood[vocab.index(s_token), classes.index(k)] = s_count / sum(sub_token_count.values())
        
for k, v in count.items():
    norm[vocab.index(k), 0] = v / NUM_OF_TOKENS


In [63]:
likelihood * np.repeat(prior,VOCAB_SIZE,axis=1).T / np.repeat(norm, NUM_OF_CLASSES, axis=1)

array([[0.90466626, 0.09664234],
       [0.8807418 , 0.11923531],
       [0.98000006, 0.02550114],
       ...,
       [1.00700407, 0.        ],
       [0.        , 0.95096063],
       [1.00700407, 0.        ]])

In [59]:
NUM_OF_TOKENS

915554